<a href="https://colab.research.google.com/github/deniskapel/ner-dialogues-hackathon/blob/master/Baseline_Flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flair

     |████████████████████████████████| 450kB 5.0MB/s 
     |████████████████████████████████| 798kB 9.9MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 1.3MB 40.7MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 983kB 37.8MB/s 
     |████████████████████████████████| 1.1MB 35.1MB/s 
     |████████████████████████████████| 890kB 40.4MB/s 
     |████████████████████████████████| 2.9MB 34.2MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.7.0-cp36-none-any.whl size=14376 sha256=ce86acf16799e1128e809ff2268f8eec7231eaf53ec410bf0acac00dca0b6038
  Stored in directory: /root/.cache/pip/wheels/cf/c6/4f/2c64a43f041415eb8b8740bd80e15e92f0d46c5e464d8e4b9b
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116678 sha256=6e91c20eab364136437979edfb6e8b486d455a57205090a3f65b7e64b97c939a
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d

In [2]:
# Uncomment if you need to download data
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/test.conll

--2021-02-09 16:37:08--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125059 (122K) [text/plain]
Saving to: ‘train.conll’

train.conll         100%[===================>] 122.13K  --.-KB/s    in 0.02s   

2021-02-09 16:37:09 (5.70 MB/s) - ‘train.conll’ saved [125059/125059]

--2021-02-09 16:37:09--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14109 (14K) [text

In [3]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings

In [4]:
import flair
import torch
# uncomment for cpu-only training
# flair.device = torch.device('cpu')

In [5]:
# 1. get the corpus
corpus: Corpus = ColumnCorpus(
    "./", column_format={0:'text', 1:'ner'},
    train_file='train.conll', dev_file='dev.conll', test_file='test.conll',
)
print(corpus)

2021-02-09 16:37:17,761 Reading data from .
2021-02-09 16:37:17,762 Train: train.conll
2021-02-09 16:37:17,763 Dev: dev.conll
2021-02-09 16:37:17,765 Test: test.conll
Corpus: 1966 train + 219 dev + 547 test sentences


In [14]:
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

# 4. initialize embeddings
embedding_types = [
    # standard word embeddings
    # WordEmbeddings('glove'),

    CharacterEmbeddings(),

    # contextual string embeddings, forward
    FlairEmbeddings('bg-forward'),

    # contextual string embeddings, backward
    FlairEmbeddings('bg-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

Dictionary with 14 tags: <unk>, O, B-SINGER, I-SINGER, B-SONG, B-FILM, I-SONG, I-FILM, B-COMPOSER, I-COMPOSER, B-BOOK, I-BOOK, <START>, <STOP>


In [16]:
# 5. initialize sequence tagger
from flair.models import SequenceTagger
from torch.optim.lr_scheduler import OneCycleLR

tagger: SequenceTagger = SequenceTagger(hidden_size=64,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
history = trainer.train(
    './models/baseline-charembeddings',
    learning_rate=0.25,
    mini_batch_size=32,
    scheduler=OneCycleLR,
    max_epochs=10
)

2021-02-09 17:23:54,877 ----------------------------------------------------------------------------------------------------
2021-02-09 17:23:54,879 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(4336, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=4336, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(4336, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=4336, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): 